In [ ]:
# default_exp layers

# Layers

> Pytorch model layers

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from mrl.imports import *
from mrl.torch_imports import *

In [ ]:
# export

class Linear(nn.Module):
    def __init__(self, d_in, d_out, act=True, bn=False, dropout=0.):
        super().__init__()
        
        layers = [nn.Linear(d_in, d_out)]
        
        if bn:
            layers.append(nn.BatchNorm1d(d_out))
            
        if act:
            layers.append(nn.ReLU())
            
        if dropout>0.:
            layers.append(nn.Dropout(p=dropout))
            
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x)

In [ ]:
# export

class LSTMBase(nn.Module):
    def __init__(self, d_vocab, d_embedding, d_hidden, n_layers, pad_idx):
        super().__init()
        
        self.d_vocab = d_vocab
        self.d_embedding = d_embedding
        self.d_hidden = d_hidden
        self.n_layers = n_layers
        self.pad_idx = pad_idx
        
        self.embedding = nn.Embedding(d_vocab, d_embedding, padding_idx=pad_idx)
        
        self.lstms = []
        
        for i in range(n_layers):
            input_size = d_embedding if i==0 else d_hidden
            output_side = d_embedding if i==n_layers-1 else d_hidden
                
            lstm = nn.LSTM(input_size, output_size, 1, batch_first=True)
            self.lstms.append(lstm)
            
        self.lstms = nn.ModuleList(self.lstms)
                
    def forward(self, x):
        x = self.embedding(x)
        
        hiddens = []
        for i, lstm in enumerate(self.lstms):
            x, (h,c) = lstm(x)
            hiddens.append((h.detach(), c.detach()))
        
        self.last_hiddens = hiddens

        return x
    
class LMHead(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        
        self.layer = nn.Linear(d_in, d_out)
        
    def forward(self, x):
        return self.layer(x)